In [56]:
import requests 
from requests import RequestException
import re
import json

In [128]:
def get_page(url):
    try:
        headers = {'accept': "application/json, text/plain, */*",
                   'Authorization': '3DF770D362-C957-83DB-0807-E9EE956366E3',
                   'X-Request-Id': '19f78a72-38c2-dc7a-45f8-4930662373c7',
                    'connection': "keep-alive",
                    'referer': "https://api.zsxq.com",
                    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",}
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            return response
        return None
    except RequestException:
        return None

In [145]:
with open('raw.txt',encoding='utf-8') as f:
    content = json.loads(f.read())
    raw_str = json.dumps(content,ensure_ascii=False)
    raw_dict = json.loads(raw_str,encoding='utf-8')
#     print(content_dict)
comments= raw_dict.get('resp_data').get("topics")
if comments:
        for _comment in comments:
            _create_time = _comment.get("create_time")
            _text = _comment.get("text").encode("utf-8")
            _id = re.match(reg_digital, _text.split("<")[0], re.M).group()
            _url = urllib.unquote(urllib.unquote(re.findall(r'href="(.*?)"', _text.split("<")[1])[0]))
            print("学号: {0} 打卡时间：{1} 链接：{2}".format(_id, _create_time, _url))


AttributeError: 'NoneType' object has no attribute 'encode'